In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.ticker import ScalarFormatter
import warnings
warnings.filterwarnings('ignore')

# Import Data

In [ ]:
# Data from https://taxfoundation.org/data/all/federal/historical-income-tax-rates-brackets/
data_file = 'Historical Income Tax Rates and Brackets, 1862-2021.csv'
df = pd.read_csv(data_file)
df.dropna(subset=['Year'], inplace=True)
df.dropna(subset=['Married Filing Jointly (Rates/Brackets)'], inplace=True)

# Remove columns
cols_to_remove = [2, 5, 8, 11]
df = df.iloc[:, [i for i in range(df.shape[1]) if i not in cols_to_remove]]

# Fix years column
df['Year'] = df['Year'].apply(lambda x: int(x[:4]))

df

# Functions for Calculating Taxes

In [ ]:
def parse_brackets(brackets):
    """
    Parse the tax brackets from string format to numerical format.
    
    :param brackets: A 2D array where the first column is strings with the % sign,
                     and the second column is strings with the $ sign.
    :return: A list of tuples where each tuple contains the upper limit of the bracket and the tax rate.
    """
    parsed_brackets = []
    for rate_str, limit_str in brackets:
        # Remove the % sign from the rate, convert to float and then to a decimal
        rate = float(rate_str.replace('%', '')) / 100
        # Remove the $ sign and commas from the limit, convert to float
        limit = float(limit_str.replace('$', '').replace(',', ''))
        parsed_brackets.append((limit, rate))
    return parsed_brackets

def calculate_income_tax(income, brackets):
    """
    Calculate income tax based on given tax brackets.
    
    :param income: The taxable income
    :param brackets: A 2D array where the first column is strings with the % sign,
                     and the second column is strings with the $ sign.
    :return: The calculated tax
    """
    parsed_brackets = np.array(parse_brackets(brackets))
    parsed_brackets[:, 0] = np.append(parsed_brackets[1:, 0], 1e8)

    brackets
    
    tax = 0
    previous_limit = 0

    for limit, rate in parsed_brackets:
        if income > limit:
            tax += (limit - previous_limit) * rate
            previous_limit = limit
        else:
            tax += (income - previous_limit) * rate
            return tax

    # For the highest bracket where income > last defined limit
    if income > previous_limit:
        tax += (income - previous_limit) * parsed_brackets[-1][1]

    return tax

In [ ]:
# Generate a geometric sequence of floating-point numbers
incomes = np.geomspace(1000, 4e6, num=100, dtype=float)

# Convert the array to a list of integers
incomes = np.array([int(x) for x in incomes])

# 2021 - Filing Method Comparison

In [ ]:
years = np.unique(df['Year'].values)

# Get tax brackets for a specific year
year = 2021
df_year = df[df['Year'] == year]

tax_brackets = {}
tax_brackets['Married Filing Jointly'] = df_year.iloc[:, [1, 2]].values
tax_brackets['Married Filing Separately'] = df_year.iloc[:, [3, 4]].values
tax_brackets['Single Filer'] = df_year.iloc[:, [5, 6]].values
tax_brackets['Head of Household'] = df_year.iloc[:, [7, 8]].values

plt.figure(figsize=(10, 8))

for label, tax_brackets in tax_brackets.items():

    taxes = np.array([calculate_income_tax(income, tax_brackets) for income in incomes])

    plt.subplot(2, 1, 1)
    plt.semilogx(incomes / 1000, taxes / incomes * 100, marker='o', markersize=0, mec='black', label=label)

    plt.subplot(2, 1, 2)
    plt.loglog(incomes / 1000, taxes / 1000, marker='o', markersize=0, mec='black', label=label)


plt.subplot(2, 1, 1)
plt.grid(True)
plt.xlabel('Income [1000 $]')
plt.ylabel('Tax Rate [%]')
plt.title('Effective Taxes vs Income')
formatter = ScalarFormatter()
formatter.set_scientific(False)
plt.gca().xaxis.set_major_formatter(formatter)
plt.gca().yaxis.set_major_formatter(formatter)
plt.legend(loc='best')

plt.subplot(2, 1, 2)
plt.grid(True)
plt.xlabel('Income [1000 $]')
plt.ylabel('Taxes [1000 $]')
plt.title('Effective Taxes vs Income')
formatter = ScalarFormatter()
formatter.set_scientific(False)
plt.gca().xaxis.set_major_formatter(formatter)
plt.gca().yaxis.set_major_formatter(formatter)
plt.legend(loc='best')

plt.tight_layout()

# Filing Single - Based on Year

In [ ]:
cutoff_year = 2002
year_interval = 2
filing_status = 'Single Filer'

years = np.unique(df['Year'].values)
years = years[years > cutoff_year][::year_interval]

plt.figure(figsize=(10, 8))

for year_idx, year in enumerate(years):

    # Get tax brackets for a specific year
    df_year = df[df['Year'] == year]

    # Break if no income tax
    if df_year.shape[0] < 2:
        continue

    tax_brackets = {}
    tax_brackets['Married Filing Jointly'] = df_year.iloc[:, [1, 2]].values
    tax_brackets['Married Filing Separately'] = df_year.iloc[:, [3, 4]].values
    tax_brackets['Single Filer'] = df_year.iloc[:, [5, 6]].values
    tax_brackets['Head of Household'] = df_year.iloc[:, [7, 8]].values
    tax_bracket = tax_brackets[filing_status]

    taxes = np.array([calculate_income_tax(income, tax_bracket) for income in incomes])

    plt.subplot(2, 1, 1)
    plt.semilogx(incomes / 1000, taxes / incomes * 100, marker='o', markersize=0, mec='black', label=year)

    plt.subplot(2, 1, 2)
    plt.loglog(incomes / 1000, taxes / 1000, marker='o', markersize=0, mec='black', label=year)

    if year_idx == 0:
        plt.subplot(2, 1, 1)
        plt.grid(True)
        plt.xlabel('Income [1000 $]')
        plt.ylabel('Effective Tax Rate [%]')
        plt.title('Effective Tax Rate vs Income')
        formatter = ScalarFormatter()
        formatter.set_scientific(False)
        plt.gca().xaxis.set_major_formatter(formatter)
        plt.gca().yaxis.set_major_formatter(formatter)

        plt.subplot(2, 1, 2)
        plt.grid(True)
        plt.xlabel('Income [1000 $]')
        plt.ylabel('Taxes [1000 $]')
        plt.title('Effective Taxes vs Income')
        formatter = ScalarFormatter()
        formatter.set_scientific(False)
        plt.gca().xaxis.set_major_formatter(formatter)
        plt.gca().yaxis.set_major_formatter(formatter)

    if year_idx == (len(years) - 1):
        plt.subplot(2, 1, 1)
        plt.legend(loc='best', ncol=3)
        plt.subplot(2, 1, 2)
        plt.legend(loc='best', ncol=3)

        plt.tight_layout()

# Based on Year - Few Incomes

In [ ]:
cutoff_year = 1850
year_interval = 1
filing_status = 'Single Filer'

incomes_few = [10000, 20000, 50000, 100000, 200000, 500000, 1000000]

years = np.unique(df['Year'].values)
years = years[years > cutoff_year][::year_interval]

df_few = pd.DataFrame()

for year_idx, year in enumerate(years):

    # Get tax brackets for a specific year
    df_year = df[df['Year'] == year]

    # Break if no income tax
    if df_year.shape[0] < 2:
        continue

    tax_brackets = {}
    tax_brackets['Married Filing Jointly'] = df_year.iloc[:, [1, 2]].values
    tax_brackets['Married Filing Separately'] = df_year.iloc[:, [3, 4]].values
    tax_brackets['Single Filer'] = df_year.iloc[:, [5, 6]].values
    tax_brackets['Head of Household'] = df_year.iloc[:, [7, 8]].values
    tax_bracket = tax_brackets[filing_status]

    taxes = np.array([calculate_income_tax(income, tax_bracket) for income in incomes_few])

    df_few[year] = taxes
df_few.index = incomes_few
df_few = df_few.T
df_few

years = df_few.index

plt.figure(figsize=(10, 4))

for income in incomes_few:

    taxes = df_few[income].values

    # plt.semilogy(years, taxes / income * 100, marker='o', markersize=0, mec='black', label=income / 1000)
    plt.plot(years, taxes / income * 100, marker='o', markersize=0, mec='black', label='%ik' % int(income / 1000))

plt.grid(True)
plt.xlabel('Year')
plt.ylabel('Effective Tax Rate [%]')
plt.title('Effective Tax Rate vs Income')
formatter = ScalarFormatter()
formatter.set_scientific(False)
# plt.gca().xaxis.set_major_formatter(formatter)
plt.gca().yaxis.set_major_formatter(formatter)
plt.legend(loc='best', ncol=1, title='Income [$]')
plt.ylim([0, 100])
# plt.xlim([1960, 1994])


In [ ]:
df[df['Year'] == 1989]

# Based on Year - Few Incomes - Inflation Adjusted

In [ ]:
cutoff_year = 1850
year_interval = 1
filing_status = 'Single Filer'

incomes_few = [10000, 20000, 50000, 100000, 200000, 500000, 1000000, 5000000]

years = np.unique(df['Year'].values)
years = years[years > cutoff_year][::year_interval]

df_few = pd.DataFrame()

for year_idx, year in enumerate(years):

    # Get tax brackets for a specific year
    df_year = df[df['Year'] == year]

    # Break if no income tax
    if df_year.shape[0] < 2:
        continue

    tax_brackets = {}
    tax_brackets['Married Filing Jointly'] = df_year.iloc[:, [1, 2]].values
    tax_brackets['Married Filing Separately'] = df_year.iloc[:, [3, 4]].values
    tax_brackets['Single Filer'] = df_year.iloc[:, [5, 6]].values
    tax_brackets['Head of Household'] = df_year.iloc[:, [7, 8]].values
    tax_bracket = tax_brackets[filing_status]

    taxes = np.array([calculate_income_tax(income, tax_bracket) for income in incomes_few])

    df_few[year] = taxes
df_few.index = incomes_few
df_few = df_few.T
df_few

years = df_few.index

plt.figure(figsize=(10, 4))

for income in incomes_few:

    taxes = df_few[income].values

    # plt.semilogy(years, taxes / income * 100, marker='o', markersize=0, mec='black', label=income / 1000)
    plt.plot(years, taxes / income * 100, marker='o', markersize=0, mec='black', label='%ik' % int(income / 1000))

plt.grid(True)
plt.xlabel('Year')
plt.ylabel('Effective Tax Rate [%]')
plt.title('Effective Tax Rate vs Income')
formatter = ScalarFormatter()
formatter.set_scientific(False)
# plt.gca().xaxis.set_major_formatter(formatter)
plt.gca().yaxis.set_major_formatter(formatter)
plt.legend(loc='best', ncol=1, title='Income [$]')
plt.ylim([0, 100])


# Effective Tax Rate vs Income [2021]

In [ ]:
cutoff_year = 1850
year_interval = 1
filing_status = 'Single Filer'

incomes_few = [10000, 20000, 50000, 100000, 200000, 500000, 1000000, 5000000]

years = np.unique(df['Year'].values)
years = years[years > cutoff_year][::year_interval]

df_few = pd.DataFrame()

for year_idx, year in enumerate(years):

    # Get tax brackets for a specific year
    df_year = df[df['Year'] == year]

    # Break if no income tax
    if df_year.shape[0] < 2:
        continue

    tax_brackets = {}
    tax_brackets['Married Filing Jointly'] = df_year.iloc[:, [1, 2]].values
    tax_brackets['Married Filing Separately'] = df_year.iloc[:, [3, 4]].values
    tax_brackets['Single Filer'] = df_year.iloc[:, [5, 6]].values
    tax_brackets['Head of Household'] = df_year.iloc[:, [7, 8]].values
    tax_bracket = tax_brackets[filing_status]

    taxes = np.array([calculate_income_tax(income, tax_bracket) for income in incomes_few])

    df_few[year] = taxes
df_few.index = incomes_few
df_few = df_few.T
df_few

years = df_few.index

year_taxes = df_few.loc[2021, :]
year_income = year_taxes.index.values
year_taxes = year_taxes.values

# Create a new array with labels
labels = []
for x in year_income:
    if x >= 1000000:
        labels.append('{:.0f}M'.format(x / 1000000))
    else:
        labels.append('{:.0f}k'.format(x / 1000))

for i in range(2):

    plt.figure(figsize=(10, 4))
    if i == 0:
        plt.plot(year_income, year_taxes / year_income * 100, marker='o', markersize=5, mec='black')
    else:
        plt.semilogx(year_income, year_taxes / year_income * 100, marker='o', markersize=5, mec='black')

    plt.grid(True)
    plt.xlabel('Income [1000 $]')
    plt.ylabel('Effective Tax Rate [%]')
    plt.title('Effective Tax Rate vs Income')

    plt.xticks(year_income, labels, rotation=45)

    # plt.xticks([year_income / 1000], ['10k', '20k',])
    plt.legend(loc='best', ncol=1, title='Income [$]')
    plt.ylim([0, 50])


# Animation

In [ ]:
cutoff_year = 1980
year_interval = 1
filing_status = 'Single Filer'

years = np.unique(df['Year'].values)
years = years[years > cutoff_year]

plt.figure(figsize=(10, 8))

for year_idx, year in enumerate(years[::year_interval]):

    # Get tax brackets for a specific year
    df_year = df[df['Year'] == year]

    # Break if no income tax
    if df_year.shape[0] < 2:
        continue

    tax_brackets = {}
    tax_brackets['Married Filing Jointly'] = df_year.iloc[:, [1, 2]].values
    tax_brackets['Married Filing Separately'] = df_year.iloc[:, [3, 4]].values
    tax_brackets['Single Filer'] = df_year.iloc[:, [5, 6]].values
    tax_brackets['Head of Household'] = df_year.iloc[:, [7, 8]].values
    tax_bracket = tax_brackets[filing_status]

    taxes = np.array([calculate_income_tax(income, tax_bracket) for income in incomes])

    plt.subplot(2, 1, 1)
    plt.semilogx(incomes / 1000, taxes / incomes * 100, marker='o', markersize=0, mec='black', label=year)

    plt.subplot(2, 1, 2)
    plt.loglog(incomes / 1000, taxes / 1000, marker='o', markersize=0, mec='black', label=year)

    if year_idx == 0:
        plt.subplot(2, 1, 1)
        plt.grid(True)
        plt.xlabel('Income [1000 $]')
        plt.ylabel('Effective Tax Rate [%]')
        plt.title('Effective Tax Rate vs Income')
        formatter = ScalarFormatter()
        formatter.set_scientific(False)
        plt.gca().xaxis.set_major_formatter(formatter)
        plt.gca().yaxis.set_major_formatter(formatter)
        plt.legend(loc='best', ncol=3)

        plt.subplot(2, 1, 2)
        plt.grid(True)
        plt.xlabel('Income [1000 $]')
        plt.ylabel('Taxes [1000 $]')
        plt.title('Effective Taxes vs Income')
        formatter = ScalarFormatter()
        formatter.set_scientific(False)
        plt.gca().xaxis.set_major_formatter(formatter)
        plt.gca().yaxis.set_major_formatter(formatter)
        plt.legend(loc='best', ncol=3)

        plt.tight_layout()